In [1]:
from biolearn.data_library import DataLibrary
import pandas as pd
# https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE19711

data = DataLibrary().get("GSE19711").load()
data.metadata, data.dnam

#elastic for selection, boost for prediction

(            age       histology
 id                             
 GSM491937  68.0    Endometrioid
 GSM491938  81.0  Carcinosarcoma
 GSM491939  56.0                
 GSM491940  62.0                
 GSM491941  72.0        Mucinous
 ...         ...             ...
 GSM492472  58.0                
 GSM492473  61.0                
 GSM492474  72.0                
 GSM492475  65.0                
 GSM492476  64.0                
 
 [540 rows x 2 columns],
             GSM491937  GSM491938  GSM491939  GSM491940  GSM491941  GSM491942  \
 id                                                                             
 cg00000292   0.756367   0.834702   0.774165   0.799517   0.819867   0.802813   
 cg00002426   0.797859   0.859538   0.769661   0.854328   0.853270   0.863897   
 cg00003994   0.068605   0.067469   0.056937   0.063802   0.061275   0.054674   
 cg00005847   0.131004   0.197486   0.140949   0.168209   0.137825   0.183501   
 cg00006414   0.076355   0.096817   0.156980   0.086761   

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

X = data.dnam.transpose()
X_df = pd.DataFrame(X)
y = data.metadata['age']
y = pd.DataFrame(y)
cb = pd.merge(X_df, y, left_index=True, right_index=True)
imputer = KNNImputer(n_neighbors=3)
cb = pd.DataFrame(imputer.fit_transform(cb), columns=cb.columns)


In [35]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_df, y, test_size=0.2, random_state=42)
X_test2 = pd.DataFrame(imputer.fit_transform(X_test2), columns=X_test2.columns)
X_test2, y_test2

(id   cg00000292  cg00002426  cg00003994  cg00005847  cg00006414  cg00007981  \
 0      0.828262    0.854375    0.059711    0.115659    0.049387    0.027008   
 1      0.734994    0.787240    0.052274    0.172818    0.075584    0.029014   
 2      0.806843    0.863183    0.055113    0.087853    0.053807    0.019662   
 3      0.784203    0.819974    0.079304    0.170066    0.072472    0.029986   
 4      0.879105    0.810259    0.087841    0.178102    0.094256    0.062319   
 ..          ...         ...         ...         ...         ...         ...   
 103    0.804033    0.823022    0.088943    0.197545    0.093083    0.035691   
 104    0.789046    0.771894    0.142923    0.148564    0.127663    0.046614   
 105    0.776728    0.788085    0.061667    0.135294    0.104459    0.039748   
 106    0.696125    0.692183    0.113258    0.170320    0.065010    0.028731   
 107    0.800268    0.849626    0.057081    0.143896    0.057264    0.027778   
 
 id   cg00008493  cg00008713  cg000094

In [3]:
# rfe for feature selection


X_imputed = cb.drop(columns=['age'])
y_imputed = cb['age']

# Feature selection using RFE with SVM
svr = SVR(kernel='linear')
rfe = RFE(estimator=svr, n_features_to_select=200, step=25)
rfe.fit(X_imputed, y_imputed)

# Get the selected features
selected_features = X_imputed.columns[rfe.support_]

# Select the top 200 features
X_selected = X_imputed[selected_features]


In [28]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y_imputed, test_size=0.2, random_state=42)

# Train an SVM model
svm_model = SVR(kernel='linear',C=10, epsilon=1.8)
svm_model.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mse, mae, r2

(15.03435020157261, 2.8406084484078655, 0.7696680969303518)

In [42]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Train an SVM model
svm_model = SVR(kernel='linear',C=10, epsilon=1.8)
svm_model.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mse, mae, r2

/Users/petra/Desktop/study/UCSF/datasci 223 python/datasci_223/.conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(15.03435020157261, 2.8406084484078655, 0.7696680969303518)

In [45]:
predictions = pd.DataFrame({
    'sampleId': y_test.index,
    'predictedAge': y_pred
})

predictions.to_csv('predictions.csv', index=False)

In [26]:
predictions = pd.DataFrame({
    'sampleId': X_test.index,
    'predictedAge': y_pred
})

predictions.to_csv('predictions.csv', index=False)

# Output for sanity check
print(predictions.head())

   sampleId  predictedAge
0       229     72.009625
1        73     59.335954
2       521     57.433693
3        86     62.354625
4       469     72.825691


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV

# Assuming X_genomic is your genomic dataset and y_age is the corresponding age labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the ElasticNetCV model
alpha_values = np.logspace(-3, 3, 10)  # Range of alpha values to search
l1_ratio_values = [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]  # Range of l1_ratio values to search
elasticnet_cv = ElasticNetCV(alphas=alpha_values, l1_ratio=l1_ratio_values, cv=5)
elasticnet_cv.fit(X_train, y_train)

# Print the selected alpha and l1_ratio values
print("Selected alpha:", elasticnet_cv.alpha_)
print("Selected l1_ratio:", elasticnet_cv.l1_ratio_)

# Evaluate the model on the test set
test_score = elasticnet_cv.score(X_test, y_test)
print("Test Set R^2 Score:", test_score)


In [ ]:
#####  feature selection
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
X_feat = X
y_feat = y
# X_feat, y = make_regression(n_samples=100, n_features=100, noise=0.1, random_state=42)
model = ElasticNet(alpha=0.00464, l1_ratio=0.1) 
model.fit(X_feat, y_feat)
feature_importance = model.coef_
sorted_indices = np.argsort(np.abs(feature_importance))[::-1]
top_features = sorted_indices[:10]
# print("Top features:")
# for idx in top_features:
#     print(f"Feature {idx}: {feature_importance[idx]}")
top_feature_names = X_feat.columns[top_features]
top_feature_coefficients = feature_importance[top_features]

# Create a DataFrame with feature names and their coefficients
feature_data = pd.DataFrame({
    'Feature': top_feature_names,
    'Coefficient': top_feature_coefficients
})

print("Top features and their coefficients:")
print(feature_data)

# Output to CSV
feature_data.to_csv('top_features.csv', index=False)

In [ ]:
choose = cb[['age', 'cg06493994','cg22736354', 'cg18219226', 'cg02228185',	'cg22580512',	'cg02288165']]

In [ ]:

preds_df = pd.DataFrame({'sampleId': X_df.index, 'predictedAge': y_pred})
preds_df.to_csv('predictions.csv', index=False)


In [ ]:
###### XGBoost

import xgboost as xgb
from sklearn.model_selection import train_test_split
#X_choose, y = make_regression(n_samples=100, n_features=100, noise=0.1, random_state=42)

# Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(choose.drop('age', axis=1), choose['age'], test_size=0.2, random_state=42)

# Train an XGBoost model
xgb_model = xgb.XGBRegressor(n_estimators=150, learning_rate=0.15, max_depth=2, reg_alpha=1, reg_lambda=0.1)
xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='rmse', early_stopping_rounds=10, verbose=False)

# Evaluate the model
train_score = xgb_model.score(X_train, y_train)
test_score = xgb_model.score(X_test, y_test)
print("XGBoost model train score:", train_score)
print("XGBoost model test score:", test_score)

In [ ]:
from sklearn.metrics import mean_squared_error

# Predictions on the training and testing sets
y_train_pred = xgb_model.predict(X_train)
y_test_pred = xgb_model.predict(X_test)

# Calculate the Mean Squared Error
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print("MSE on training set:", mse_train)
print("MSE on testing set:", mse_test)

from sklearn.metrics import r2_score, roc_auc_score

# Calculate R-squared (R2) score
r2 = r2_score(y_test, y_test_pred)
print("R-squared (R2) score:", r2)


In [ ]:
#########random forest

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],  # Number of trees in the forest
    'max_depth': [None, 10, 20],      # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]     # Minimum number of samples required to be at a leaf node
}

# Instantiate the Random Forest regressor
rf_model = RandomForestRegressor(random_state=42)

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_rf_model = grid_search.best_estimator_

print("Best parameters:", best_params)

# Evaluate the best model
y_train_pred = best_rf_model.predict(X_train)
y_test_pred = best_rf_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print("MSE on training set with best model:", mse_train)
print("MSE on testing set with best model:", mse_test)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Instantiate a new Random Forest model with the best parameters
best_rf_model = RandomForestRegressor(n_estimators=150,
                                       #max_depth=5,
                                       min_samples_split=2,
                                       min_samples_leaf=4,
                                       random_state=42)

# Fit the model to the training data
best_rf_model.fit(X_train, y_train)

# Evaluate the model on the testing data
y_test_pred = best_rf_model.predict(X_test)
mse_test = mean_squared_error(y_test, y_test_pred)

print("MSE on testing set with best model:", mse_test)


In [ ]:
cb[['age', 'cg06493994','cg22736354', 'cg18219226', 'cg02228185',	'cg22580512',	'cg02288165', 'cg22549408','cg05740244','cg00019495']].to_csv('selected.csv', index=False)
